In [1]:
import torch
from transformers import pipeline
import soundfile as sf
from tqdm import tqdm
import logging
from pathlib import Path

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def transcribe_wav(wav_path, model_name="openai/whisper-tiny.en", device=None, task='transcribe', batch_size=8):
    """
    Transcribe a WAV audio file using Whisper with Hugging Face pipeline and return a formatted string.
    
    Parameters:
        wav_path (str or Path): Path to the input WAV audio file.
        model_name (str): Hugging Face model name.
        device (str or int, optional): Device to run the model on ('cpu', 'cuda:0', etc.). Defaults to automatic selection.
        task (str): Task type ('transcribe' or 'translate').
        batch_size (int): Batch size for processing.
    
    Returns:
        str: Formatted transcription string with timestamps.
    """
    try:
        # Determine device
        if device is None:
            device = "cuda:0" if torch.cuda.is_available() else "cpu"

        logging.info(f"Using device: {device}")

        # Initialize the pipeline
        logging.info(f"Loading model '{model_name}' for task '{task}'.")
        pipe = pipeline(
            "automatic-speech-recognition",
            model=model_name,
            chunk_length_s=30,
            device=device,
        )

        # Load the WAV file
        wav_path = Path(wav_path)
        if not wav_path.exists():
            raise FileNotFoundError(f"Audio file not found: {wav_path}")
        
        logging.info(f"Loading audio file: {wav_path}")
        audio_input, sr = sf.read(wav_path)
        inputs = {
            "raw": audio_input,
            "sampling_rate": sr,
        }

        # Perform transcription with timestamps
        logging.info("Running transcription...")
        transcription_chunks = pipe(
            inputs,
            batch_size=batch_size,
            return_timestamps=True
        ).get("chunks", [])

        logging.info("Transcription completed.")

        # Format transcription into desired string
        formatted_transcription = format_transcription(transcription_chunks)

        return formatted_transcription

    except FileNotFoundError as fnf_error:
        logging.error(f"Error: {fnf_error}")
        return ""
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        return ""

def format_transcription(transcription_chunks):
    """
    Format transcription chunks into the desired string format.

    Each line contains:
    Transcribed text | start: X | end: Y

    Parameters:
        transcription_chunks (list): List of transcription chunks with timestamps.

    Returns:
        str: Formatted transcription string.
    """
    formatted_output = ""
    for segment in tqdm(transcription_chunks, desc="Formatting transcription"):
        text = segment.get("text", "").strip()
        start, end = segment.get("timestamp", (0.0, 0.0))
        # Round the timestamps to two decimal places
        start_rounded = round(start, 2)
        end_rounded = round(end, 2) if end else round(start + 5.0, 2)  # Default to start + 5.0 if end is None
        formatted_output += f"{text} | start: {start_rounded} | end: {end_rounded}\n"
    return formatted_output



# Example usage:
transcription_text = transcribe_wav("output/audio.wav")
print(transcription_text)


2024-09-27 08:56:14,035 - INFO - Using device: cpu
2024-09-27 08:56:14,036 - INFO - Loading model 'openai/whisper-tiny.en' for task 'transcribe'.
2024-09-27 08:56:14,815 - INFO - Loading audio file: output/audio.wav
2024-09-27 08:56:14,912 - INFO - Running transcription...
/home/intelaipc/intel/oneapi/intelpython/envs/tiktalk/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
2024-09-27 08:56:17,723 - INFO - Transcription completed.
Formatting transcription: 100%|██████████| 17/17 [00:00<00:00, 165821.32it/s]

Imagine a world where humanity has finally reached the stars and we're not just talking | start: 0.0 | end: 5.28
about any old spacecraft, but the most advanced cutting edge and sustainable vessels that are | start: 5.28 | end: 11.64
changing the game. | start: 11.64 | end: 13.68
Space ships are not just a luxury, they are a necessity. | start: 13.68 | end: 17.04
They are the key to unlocking new frontiers, new discoveries and new possibilities for humanity. | start: 17.04 | end: 23.36
But did you know that the first spaceship | start: 23.36 | end: 25.52
was actually a hot air balloon? Yes, you heard that right. In 1783, French inventor | start: 25.52 | end: 32.74
Montgolfier created the first successful hot air balloon which carried a group of 20 people | start: 32.74 | end: 38.64
to the skies. It was a groundbreaking achievement that paved the way for the development of modern space travel. | start: 38.64 | end: 45.92
Fast forward to today, and we have reusable rockets, advanced prop